In [ ]:
%%capture
import ROOT
import glob
import math
import sys
import numpy as np
import pandas as pd
from IPython.display import display, Markdown, HTML
import ipywidgets as widgets
from TPCQCVis.src.drawHistograms import *
from TPCQCVis.src.drawTrending import *
from TPCQCVis.src.drawMultiTrending import *
from TPCQCVis.src.checkHistograms import *
from TPCQCVis.src.checkTrending import *
from TPCQCVis.src.utility import *
import warnings
from copy import copy
warnings.filterwarnings('ignore')

In [ ]:
%jsroot on
display(HTML("<style>.container { width:95% !important; align-items: center;}</style>"))
display(HTML("<style>table {float:left;}</style>"))
ROOT.gErrorIgnoreLevel = ROOT.kFatal
#display(HTML('<style>{}</style>'.format(CSS)))
#ROOT.gStyle.SetPalette(57)

In [ ]:
def drawQuantileProjection(objectName,rootDataFile,quantileOrder=0.5, axis="x"):
    from copy import copy
    canvas = ROOT.TCanvas()
    legend = ROOT.TLegend()
    [hist,leg,canvo,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,
                                               legendNames=runList,pads=True,drawOption="COLZ",maxColumns=3)
    quants = []
    canvas.cd()
    ROOT.gPad.SetGridy(1)
    for i,histo in enumerate(hist):
        if axis == "y":
            quant = copy(histo.QuantilesY(quantileOrder))
        else:
            quant = copy(histo.QuantilesX(quantileOrder))
        quant.SetYTitle("Median "+ histo.GetYaxis().GetTitle())
        quant.SetXTitle(histo.GetXaxis().GetTitle())
        quant.SetTitle(quant.GetYaxis().GetTitle()+" vs. "+quant.GetXaxis().GetTitle())
        quant.SetLineWidth(2)
        quants.append(quant)
        quants[i].SetStats(0)
        quants[i].Draw("SAME L")
        legend.AddEntry(quant,runList[i])
    return quants,legend,canvas
    
def sliceAndFit(objectName,rootDataFile,fitFunc="gaus",fitRange=[40,60]):
    from copy import copy
    canvas = ROOT.TCanvas()
    legend = ROOT.TLegend()
    [hist,leg,canvo,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,
                                               legendNames=runList,pads=True,drawOption="COLZ",maxColumns=3)
    fits = []
    myFunc = ROOT.TF1("myFunc","gaus",40,60)
    canvas.cd()
    for i,histo in enumerate(hist):
        try:
            histo.FitSlicesY(myFunc, 0, -1, 0, "QNR")
            fit = copy(ROOT.gDirectory.Get(histo.GetName()+"_1"))
        except:
            print("Could not perform fit.")
            return [],legend,canvas
        fit.SetYTitle("Gaus Fit Mean "+ histo.GetYaxis().GetTitle())
        fit.SetXTitle(histo.GetXaxis().GetTitle())
        fit.SetTitle(quant.GetYaxis().GetTitle()+" vs. "+quant.GetXaxis().GetTitle())
        fit.SetLineWidth(2)
        fits.append(fit)
        legend.AddEntry(fit,runList[i])
        fit.Draw("SAME L")
    legend.Draw()
    return fits, legend, canvas

In [ ]:
ROOT.gStyle.SetPalette(57)
ROOT.gStyle.SetPalette(55)
ROOT.gStyle.SetGridStyle(3)
ROOT.gStyle.SetGridWidth(1)
ROOT.gStyle.SetOptStat(0)
cols = list(ROOT.TColor.GetPalette())
def updateColors(histograms,palette):
    colors = []
    for i,hist in enumerate(histograms):
        if len(histograms)>1:
            color = palette[math.floor((i/(len(histograms)-1))*(len(palette)-1))]
        else:
            color = palette[0]
        #color = i
        colors.append(color)
        hist.SetLineColor(color)
        hist.SetMarkerColor(color)
    return colors

In [ ]:
# Notebook variables
periodName = "myPeriod"
passName = "myPass"
runNumber = 123456
path = "myPath"

#periodName = "LHC23r"
#passName = "apass3"
#runNumber = "536545"
#path = "/cave/alice-tpc-qc/data/2023/"


# Read the Root Files
fileList = glob.glob(path+"/"+periodName+"/"+passName+"/"+"*_QC.root")
fileList = [file for file in fileList if file[-13] != "_"]
fileList.sort()
#fileList = fileList[13:]
runList = [fileList[i][-14:-8] for i in range(len(fileList))]
rootDataFile=[]
for file in fileList:
    rootDataFile.append(ROOT.TFile.Open(file,"READ"))

# TPC Async QC Report - myPeriod - myPass
### RCT Tables:
+ [2022](https://docs.google.com/spreadsheets/d/14vXFYVx3oVE8wgJKmIBhjvAt6NpGp7D6H4AmBM9E0Cw/edit#gid=0), [2023](https://docs.google.com/spreadsheets/d/1YBQLXWwwc3aC3B_PYVpFkTgEP0n6u1ovtYfiCOMWnTc/edit?pli=1#gid=0), [2023_PbPb](https://docs.google.com/spreadsheets/d/1vsl-pkLdUoNXZm6muPyMENWdDSKM7UDtQLcXc2B9QhE/edit#gid=492527515)

In [ ]:
import datetime
now = datetime.datetime.now()
display(Markdown("+ Report automatically created on "+now.strftime("%d.%m.%Y at %H:%M:%S")))

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#TPC-asyncQC" data-toc-modified-id="TPC-asyncQC">TPC asyncQC</a></span><ul class="toc-item"><li><span><a href="#General-notes" data-toc-modified-id="General-notes-1.1">General notes</a></span></li><li><span><a href="#Run-Table" data-toc-modified-id="Run-Table-1.2">Run Table</a></span></li></ul></li><li><span><a href="#Quality-Observer" data-toc-modified-id="Quality-Observer-2">Quality Observer</a></span><ul class="toc-item"><li><span><a href="#Clusters" data-toc-modified-id="Clusters-2.1">Clusters</a></span></li><li><span><a href="#Tracks" data-toc-modified-id="Tracks-2.2">Tracks</a></span></li></ul></li><li><span><a href="#Trendings" data-toc-modified-id="Trendings-3">Trendings</a></span></li><li><span><a href="#Plots" data-toc-modified-id="Plots-4">Plots</a></span><ul class="toc-item"><li><span><a href="#Clusters" data-toc-modified-id="Clusters-4.1">Clusters</a></span></li><li><span><a href="#Tracks" data-toc-modified-id="Tracks-4.2">Tracks</a></span><ul class="toc-item"><li><span><a href="#Geometrical-distributions-of-tracks" data-toc-modified-id="Geometrical-distributions-of-tracks-4.2.1">Geometrical distributions of tracks</a></span></li><li><span><a href="#Track-properties" data-toc-modified-id="Track-properties-4.2.2">Track properties</a></span></li></ul></li><li><span><a href="#PID" data-toc-modified-id="PID-4.3">PID</a></span><ul class="toc-item"><li><span><a href="#TPC-Gain-calibration" data-toc-modified-id="TPC-Gain-calibration-4.3.1">TPC Gain calibration</a></span></li><li><span><a href="#dEdx-vs-p" data-toc-modified-id="dEdx-vs-p-4.3.2">dEdx vs p</a></span></li></ul></li></ul></li></ul></div>

---
## Quality Observer

### Clusters

In [ ]:
qualityDFClusters = pd.DataFrame({'runNumber':runList})
names = []
for i in range(72):
    if i < 18*2: roc = "IROC-"
    else: roc = "OROC-"
    if i%(18*2) < 18: side = "A"
    else : side = "C"
    names.append(roc+side+str(i%18).zfill(2))

In [ ]:
%%capture
objectName="c_ROCs_N_Clusters_1D;1"
trending = "mean"
error = ""
[trends,canvas] = drawMultiTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,pads=True,normalize=True,
                               log="none",yAxisRange=[0,500])
for iPad,pad in enumerate(canvas.GetListOfPrimitives()):
    trend = trends[iPad]
    [qualities,pad] = checkTrending(trend,canvas=pad,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
    qualityDFClusters[names[iPad]] = qualities
    canvas.Update()
#canvas.Draw()

In [ ]:
for i,row in qualityDFClusters.iterrows():
    probs = [columnName for (columnName, columnData) in row.items() if columnData in ["MEDIUM","BAD"]]
    if len(probs) != 0:
        print([row.runNumber]+probs)

In [ ]:
%jsroot on
myPalette = np.array([920, 414, 801, 633],dtype=np.int32)
ROOT.gStyle.SetPalette(4,myPalette)
ROOT.gStyle.SetGridStyle(1)
qualityLabels = {"NULL":0,"GOOD":1,"MEDIUM":2,"BAD":3}

canvas = ROOT.TCanvas("QualityMatrixClusters","",1100,600)
canvas.SetLeftMargin(0.15)
canvas.SetBottomMargin(0.15)
canvas.SetRightMargin(0.15)
canvas.SetGrid()

qualityHistClusters = ROOT.TH2I("QualityMatrixClusters","Clusters - Quality Matrix",
                        len(qualityDFClusters.index),min(qualityDFClusters.index),max(qualityDFClusters.index)+1,
                        len(qualityDFClusters.columns)-1,0,len(qualityDFClusters.columns)-1)
qualityHistClusters.SetCanExtend(ROOT.TH1.kAllAxes)
qualityHistClusters.SetStats(0)
for runIndex,run in enumerate(qualityDFClusters.runNumber):
    qualityHistClusters.GetXaxis().SetBinLabel(runIndex+1,str(run))
    qualityHistClusters.GetXaxis().SetTickLength( 0.03)
    for checkIndex,check in enumerate(qualityDFClusters.loc[:, qualityDFClusters.columns != "runNumber"].columns):
        #print(checkIndex,runIndex)
        qualityHistClusters.Fill(run,check,qualityLabels.get(qualityDFClusters.iloc[runIndex][check]))

qualityHistClusters.LabelsOption("u")
qualityHistClusters.Draw("COLZ")
qualityHistClusters.GetZaxis().SetRangeUser(-0.5,3.5);
qualityHistClusters.GetZaxis().SetTitle("Quality")
qualityHistClusters.LabelsOption("v")
canvas.Draw()

### Tracks

In [ ]:
%%capture
qualityDF = pd.DataFrame({'runNumber':runList})
objects = ["hPhiAside;1","hPhiCside;1","hEta;1","hPt;1","hSign;1","hdEdxTotMIP_TPC;1","hNClustersAfterCuts;1","hdEdxVsTgl;1","hdEdxTotMIPVsSnp_TPC;1"]
trending = "mean"
error = "meanError"
for objectName in objects:
    if objectName == "hdEdxTotMIP_TPC;1":
        trending = "fit(gaus,Sq,N,40,60)"
    else:
        trending = "mean"
    [trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                                   trend=trending,error=error,log="none",axis=1)
    [qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
    qualityDF[objectName] = qualities

In [ ]:
canvas = ROOT.TCanvas("QualityMatrixTracks","",1000,500)
canvas.SetLeftMargin(0.15)
canvas.SetBottomMargin(0.15)
canvas.SetRightMargin(0.15)
canvas.SetGrid()

qualityHist = ROOT.TH2I("QualityMatrixTracks","Tracks Observables - Quality Matrix",
                        len(qualityDF.index),min(qualityDF.index),max(qualityDF.index)+1,
                        len(qualityDF.columns)-1,0,len(qualityDF.columns)-1)
qualityHist.SetCanExtend(ROOT.TH1.kAllAxes)
qualityHist.SetStats(0)
for runIndex,run in enumerate(qualityDF.runNumber):
    qualityHist.GetXaxis().SetBinLabel(runIndex+1,str(run))
    qualityHist.GetXaxis().SetTickLength( 0.03)
    for checkIndex,check in enumerate(qualityDF.loc[:, qualityDF.columns != "runNumber"].columns):
        #print(checkIndex,runIndex)
        qualityHist.Fill(run,check,qualityLabels.get(qualityDF.iloc[runIndex][check]))

qualityHist.LabelsOption("u")
qualityHist.Draw("COLZ")
qualityHist.GetZaxis().SetRangeUser(-0.5,3.5)
qualityHist.GetZaxis().SetTitle("Quality")
qualityHist.LabelsOption("v")

# Update the canvas
canvas.Update()
canvas.Draw()

---
## Trendings
### Mean phi of tracks

In [ ]:
%jsroot on
objectName="hPhiAside;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,log="none")
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
canvas.SetCanvasSize(1000,400)
qualityDF[objectName] = qualities
canvas.Draw()

In [ ]:
%jsroot on
objectName="hPhiCside;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,log="none")
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
qualityDF[objectName] = qualities
canvas.SetCanvasSize(1000,400)
#trend.SetTitle("Number of tracks - Trend")
canvas.Draw()

### Mean eta of tracks

In [ ]:
%jsroot on
objectName="hEta;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,log="none")
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
qualityDF[objectName] = qualities
#trend.SetTitle("Number of tracks - Trend")
canvas.SetCanvasSize(1000,400)
canvas.Draw()

### Mean tracks transverse momentum of tracks

In [ ]:
%jsroot on
objectName="hPt;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,log="none")
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
qualityDF[objectName] = qualities
#trend.SetTitle("Number of tracks - Trend")
canvas.SetCanvasSize(1000,400)
canvas.Draw()

### Mean charge sign of tracks

In [ ]:
%jsroot on
objectName="hSign;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,log="none")
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
qualityDF[objectName] = qualities
#trend.SetTitle("Number of tracks - Trend")
canvas.SetCanvasSize(1000,400)
canvas.Draw()

### Mean dEdxTot of MIPs tracks

In [ ]:
%jsroot on
objectName="hdEdxTotMIP_TPC;1"
trending = "fit(gaus,Sq,N,40,60)"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,log="none",yAxisRange=[30,70])
trend.Draw()
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})

qualityDF[objectName] = qualities
#trend.GetYaxis().SetRangeUser(45,55)
#trend.SetTitle("Number of tracks - Trend")
canvas.SetCanvasSize(1000,400)
canvas.Draw()

**Mean number of clusters per tracks:**

In [ ]:
%jsroot on
objectName="hNClustersAfterCuts;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,log="none")
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
qualityDF[objectName] = qualities
#trend.SetTitle("Number of tracks - Trend")
canvas.SetCanvasSize(1000,400)
canvas.Draw()

**Mean tan(lambda) of tracks:**

In [ ]:
%jsroot on
objectName="hdEdxVsTgl;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,log="none")
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
qualityDF[objectName] = qualities
#trend.SetTitle("Number of tracks - Trend")
canvas.SetCanvasSize(1000,400)
canvas.Draw()

**Mean sin(phi) of tracks:**

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsSnp_TPC;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,log="none")
[qualities,canvas] = checkTrending(trend,canvas=canvas,thresholds={"GOOD":1.5,"MEDIUM":3,"BAD":6})
qualityDF[objectName] = qualities
#trend.SetTitle("Number of tracks - Trend")
canvas.SetCanvasSize(1000,400)
canvas.Draw()

---
## Plots

In [ ]:
ROOT.gStyle.SetPalette(57)
ROOT.gStyle.SetGridStyle(3)
ROOT.gStyle.SetGridWidth(1)

### Clusters
**Number of clusters per pad, A-side:**

In [ ]:
%jsroot on

objectName="h2DRPhinClustersAside"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,
                                               legendNames=runList,pads=True,drawOption="COLZ")
    canvas.Draw()

**Number of clusters per pad, C-side:**

In [ ]:
%jsroot on
objectName="h2DRPhinClustersCside"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,legendNames=runList,
                                               pads=True,drawOption="COLZ")
    canvas.Draw()

---
**Cluster total charge per pad, A-side:**

In [ ]:
%jsroot on
objectName="h2DRPhiqTotAside"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,
                                               legendNames=runList,pads=True,drawOption="COLZ")
    canvas.Draw()

**Cluster total charge per pad, C-side:**

In [ ]:
%jsroot on
objectName="h2DRPhiqTotCside"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,
                                               legendNames=runList,pads=True,drawOption="COLZ")
    canvas.Draw()

---
### Tracks
#### Geometrical distributions of tracks
**Phi distribution A-side tracks:**

In [ ]:
%jsroot on
objectName="hPhiAside"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=runList,pads=False,
                                               drawOption="SAME L",yAxisRange=[2.2e-3,3.2e-3],grid=False)
    #Sector boundatries
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,hist[0].GetMinimum(),(j*2*math.pi)/18,hist[0].GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,hist[0].GetMaximum(),str(j))
    pad1.SetGridy(1)
    canvas.SetCanvasSize(1000,400)
    updateColors(hist,cols)
    canvas.Draw()

**Phi distribution C-side tracks:**

In [ ]:
%jsroot on
objectName="hPhiCside"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=runList,pads=False,
                                               drawOption="SAME L",yAxisRange=[2.2e-3,3.2e-3],grid=False)
    updateColors(hist,cols)
    #Sector boundatries
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,hist[0].GetMinimum(),(j*2*math.pi)/18,hist[0].GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,hist[0].GetMaximum(),str(j))
    pad1.SetGridy(1)
    canvas.SetCanvasSize(1000,400)
    canvas.Draw()

**Eta distribution:**

In [ ]:
%jsroot on
objectName="hEta"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=runList,pads=False,
                                               drawOption="SAME L",xAxisRange=[-1.1,1.1],grid=True)
    updateColors(hist,cols)
    pad1.SetGrid(1)
    canvas.SetCanvasSize(700,400)
    canvas.Draw()

In [ ]:
%jsroot on
objectName="h2DEtaPhi"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas.SetGridy(1)
    canvas.SetCanvasSize(1000,400)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(-.1,.1)
        quant.SetTitle("Median eta vs phi")
    canvas.cd()
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="h2DEtaPhi"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,yAxisRange = [-1.1,1.1],
                                               legendNames=runList,pads=True,drawOption="COLZ")
    canvas.Draw()

#### Track properties
**Transverse momentum**

In [ ]:
%jsroot on
objectName="hPt"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=runList,pads=False,
                                               drawOption="SAME L",log="logxy",
                                               grid=True)
    updateColors(hist,cols)
    pad1.SetGrid(1)
    canvas.SetCanvasSize(700,400)
    canvas.Draw()

**Track sign**

In [ ]:
%jsroot on
objectName="hSign"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=runList,pads=False,
                                               drawOption="SAME HIST",grid=True)
    updateColors(hist,cols)
    pad1.SetGrid(1)
    canvas.SetCanvasSize(700,400)
    canvas.Draw()

**Charge / transverse momentum**

In [ ]:
%jsroot on
objectName="hQOverPt"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=runList,pads=False,
                                                        drawOption="SAME L",grid=True)
    updateColors(hist,cols)
    pad1.SetGrid(1)
    canvas.SetCanvasSize(700,400)
    canvas.Draw()

**Number of clusters per track**

In [ ]:
%jsroot on
objectNames=["crossedRows","foundClusters","sharedClusters"]
if any([checkIfExists(rootDataFile,objectName) for objectName in objectNames]):
    bigCanvas = ROOT.TCanvas("","",1200,400)
    bigCanvas.Divide(3,1)
    [hist1,legend1,canvas1,pad1] = drawHistograms(objectNames[0],rootDataFile,xAxisRange = [50,160],normalize=True,legend=True,legendNames=runList)
    [hist2,legend2,canvas2,pad2] = drawHistograms(objectNames[1],rootDataFile,xAxisRange = [50,160],normalize=True,legend=True,legendNames=runList)
    [hist3,legend3,canvas3,pad3] = drawHistograms(objectNames[2],rootDataFile,xAxisRange = [0,160],normalize=True,legend=True,legendNames=runList)
    #canvas.SetCanvasSize(700,400)
    #hist.SetGrid(1)
    updateColors(hist1,cols)
    updateColors(hist2,cols)
    updateColors(hist3,cols)
    bigCanvas.cd(1)
    ROOT.gPad.SetGrid(1)
    for hist in hist1 : hist.Draw("SAME L")
    legend1.Draw()
    bigCanvas.cd(2)
    ROOT.gPad.SetGrid(1)
    for hist in hist2 : hist.Draw("SAME L")
    legend2.Draw()
    bigCanvas.cd(3)
    ROOT.gPad.SetGrid(1)
    for hist in hist3 : hist.Draw("SAME L")
    ROOT.gPad.SetLogy()
    legend3.Draw()
    bigCanvas.Draw()

In [ ]:
%jsroot on
objectName="hNClustersAfterCuts"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=runList,pads=False,
                                               drawOption="SAME L",xAxisRange = [50,160],yAxisRange=[0,40e-3],grid=True)
    updateColors(hist,cols)
    pad1.SetGrid(1)
    canvas.SetCanvasSize(700,400)
    canvas.Draw()

**nClusters vs phi**

In [ ]:
%jsroot on
objectName="h2DNClustersPhiAside"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas.SetGridy(1)
    canvas.SetCanvasSize(1000,400)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(60,160)
        quant.SetTitle("A-Side - nClusters per track vs phi")
    canvas.cd()
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
canvas.Draw()

In [ ]:
%jsroot on
objectName="h2DNClustersPhiCside"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas.SetGridy(1)
    canvas.SetCanvasSize(1000,400)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(60,160)
        quant.SetTitle("C-Side - nClusters per track vs phi")
    canvas.cd()
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
canvas.Draw()

**nClusters vs eta**

In [ ]:
%jsroot on
objectName="h2DNClustersEta"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas.SetGrid(1)
    canvas.SetCanvasSize(700,400)
    for quant in quants:
        quant.GetXaxis().SetRangeUser(-1.1,1.1)
        quant.GetYaxis().SetRangeUser(60,160)
    legend.Draw()
canvas.Draw()

**nClusters vs transverse momentum**

In [ ]:
%jsroot on
objectName="h2DNClustersPt"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas.SetGrid(1)
    canvas.SetCanvasSize(700,400)
    canvas.SetLogx()
    for quant in quants:
        quant.GetYaxis().SetRangeUser(60,160)
    legend.Draw()
canvas.Draw()

### DCAr vs phi

In [ ]:
%jsroot on
objectName="hDCAr_A_Pos"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas.SetGridy(1)
    canvas.SetCanvasSize(1000,350)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle("DCAr A-Side - Positive")
    canvas.cd()
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCAr_A_Neg"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas.SetGridy(1)
    canvas.SetCanvasSize(1000,350)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle("DCAr A-Side - Negative")
    canvas.cd()
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCAr_C_Pos"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas.SetGridy(1)
    canvas.SetCanvasSize(1000,350)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle("DCAr C-Side - Positive")
    canvas.cd()
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCAr_C_Neg"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas.SetGridy(1)
    canvas.SetCanvasSize(1000,350)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle("DCAr C-Side - Negative")
    canvas.cd()
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
    canvas.Draw()

### DCAr vs eta

In [ ]:
%jsroot on
objectName="hDCArVsEtaPos"
title = "DCAr vs eta - Positive"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas.SetGridy(1)
    canvas.SetCanvasSize(1000,350)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.GetXaxis().SetRangeUser(-1,1)
        quant.SetTitle(title)
    canvas.cd()
    legend.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCArVsEtaNeg"
title = "DCAr vs eta - Negative"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas.SetGridy(1)
    canvas.SetCanvasSize(1000,350)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.GetXaxis().SetRangeUser(-1,1)
        quant.SetTitle(title)
    canvas.cd()
    legend.Draw()
    canvas.Draw()

### DCAr vs pT

In [ ]:
%jsroot on
objectName="hDCArVsPtPos"
title = "DCAr vs pT - Positive"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas.SetGridy(1)
    canvas.SetCanvasSize(1000,350)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.GetXaxis().SetRangeUser(0.1,20)
        quant.SetTitle(title)
    canvas.cd()
    legend.Draw()
    canvas.SetLogx()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCArVsPtNeg"
title = "DCAr vs pT - Negative"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas.SetGridy(1)
    canvas.SetCanvasSize(1000,350)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.GetXaxis().SetRangeUser(0.1,20)
        quant.SetTitle(title)
    canvas.cd()
    legend.Draw()
    canvas.SetLogx()
    canvas.Draw()

### DCAr vs nClusters

In [ ]:
%jsroot on
objectName="hDCArVsNClsPos"
title = "DCAr vs nClusters - Positive"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas.SetGridy(1)
    canvas.SetCanvasSize(1000,350)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.GetXaxis().SetRangeUser(60,160)
        quant.SetTitle(title)
    canvas.cd()
    legend.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCArVsNClsNeg"
title = "DCAr vs nClusters - Negative"
if checkIfExists(rootDataFile,objectName):
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas.SetGridy(1)
    canvas.SetCanvasSize(1000,350)
    for quant in quants:
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.GetXaxis().SetRangeUser(60,160)
        quant.SetTitle(title)
    canvas.cd()
    legend.Draw()
    canvas.Draw()

### PID
#### dEdxTot of MIP

In [ ]:
%jsroot on
yRange = [0,0.09]
objects = ["hdEdxTotMIP_" + region for region in ["TPC","IROC","OROC1","OROC2","OROC3"]]
if all([checkIfExists(rootDataFile,objectName) for objectName in objects]):
    hists = []
    for objectName in objects:
        [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=runList,pads=False,
                                               drawOption="SAME L",yAxisRange=yRange,log="logx")
        updateColors(hist,cols)
        hists.append(copy(hist))
    c = ROOT.TCanvas("MIP","MIP",1100,600)
    c.Divide(2)
    leftPad = c.cd(1)
    leftPad.SetGrid(1)
    for hist in hists[0]:
        hist.Draw("SAME L")
        hist.SetStats(0)
    #Draw target 50
    targets = [ROOT.TLine(50,yRange[0],50,yRange[1]) for hist in hists]
    for i in range(len(objects)):
        targets[i].SetLineStyle(2)
        targets[i].SetLineColor(3)
        targets[i].SetLineWidth(3)
    targets[0].Draw()
    legend.Draw()
    rightPad = c.cd(2)
    rightPad.Divide(2,2)
    for i in range(1,5):
        rightPad.cd(i)
        ROOT.gPad.SetGrid(1)
        for hist in hists[i]:
            hist.Draw("SAME L")
            hist.SetStats(0)
        targets[i].Draw()
    c.Draw()

#### TPC Gain calibration
**dEdx of MIP vs sector**

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsSec_TPC"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("MIP","MIP",1100,600)
    c.Divide(2)
    [fits,legend,canvas] = sliceAndFit(objectName,rootDataFile)
    updateColors(fits,cols)
    canvas.SetGridy(1)
    canvas.SetCanvasSize(1000,350)
    for fit in fits:
        fit.GetYaxis().SetRangeUser(45,55)
        fit.SetTitle("TPC Mean MIP Q_{Tot} vs Sector")
    canvas.Draw()

**dEdx of Mip vs nClusters**

In [ ]:

%jsroot on
objectName="hdEdxTotMIPVsNcl_TPC"
if checkIfExists(rootDataFile,objectName):
    [fits,legend,canvas] = sliceAndFit(objectName,rootDataFile)
    updateColors(fits,cols)
    canvas.SetGridy(1)
    canvas.SetCanvasSize(1000,350)
    for fit in fits:
        fit.GetYaxis().SetRangeUser(45,55)
        fit.GetXaxis().SetRangeUser(60,160)
        fit.SetTitle("TPC Mean MIP Q_{Tot} vs nClusters")
        sectorBoundary = ROOT.TLine()
        sectorBoundary.SetLineStyle(2)
        sectorBoundary.SetLineColor(ROOT.kBlack)
        sectorBoundary.SetLineWidth(2)
        sectorBoundary.DrawLine(60,50,153,50)
    canvas.Draw()

**dEdx of Mip vs tan(lambda)**

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsTgl_TPC"
if checkIfExists(rootDataFile,objectName):
    [fits,legend,canvas] = sliceAndFit(objectName,rootDataFile,fitRange=[40,60])
    updateColors(fits,cols)
    canvas.SetGridy(1)
    canvas.SetCanvasSize(1000,350)
    for fit in fits:
        fit.GetYaxis().SetRangeUser(45,55)
        fit.GetXaxis().SetRangeUser(-2,2)
        fit.SetTitle("TPC Mean MIP Q_{Tot} vs tan(lambda)")
        sectorBoundary = ROOT.TLine()
        sectorBoundary.SetLineStyle(2)
        sectorBoundary.SetLineColor(ROOT.kBlack)
        sectorBoundary.SetLineWidth(2)
        sectorBoundary.DrawLine(-2,50,2,50)
    canvas.Draw()

**dEdx of Mip vs sin(phi)**

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsSnp_TPC"
if checkIfExists(rootDataFile,objectName):
    [fits,legend,canvas] = sliceAndFit(objectName,rootDataFile,fitRange=[40,60])
    updateColors(fits,cols)
    canvas.SetGridy(1)
    canvas.SetCanvasSize(1000,350)
    for fit in fits:
        fit.GetYaxis().SetRangeUser(35,65)
        fit.GetXaxis().SetRangeUser(-1,1)
        fit.SetTitle("TPC Mean MIP Q_{Tot} vs sin(phi)")
        sectorBoundary = ROOT.TLine()
        sectorBoundary.SetLineStyle(2)
        sectorBoundary.SetLineColor(ROOT.kBlack)
        sectorBoundary.SetLineWidth(2)
        sectorBoundary.DrawLine(-1,50,1,50)
    canvas.Draw()

#### dEdx vs p

In [ ]:
%jsroot on
from TPCQCVis.src.drawBetheBloch import *
objectName="hdEdxTotVsP_TPC"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,log="logxyz",legendNames=runList,pads=True,drawOption="COLZ")
    betheBlochLines = getBetheBloch(runList)
    pad1 = drawBetheBloch(betheBlochLines,pad1)
    canvas.Draw()